In [1]:
import pandas as pd
import sqlite3
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
conn=sqlite3.connect('ritu.db')

In [3]:
# returning all records from ques-posts
ques_query="SELECT * FROM [posts]"

In [4]:
post_df = pd.read_sql_query(ques_query, conn)

In [5]:
post_df.shape

(8438, 21)

In [6]:
post_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8438 entries, 0 to 8437
Data columns (total 21 columns):
Body                     8438 non-null object
ViewCount                3548 non-null float64
LastEditorDisplayName    8 non-null object
ClosedDate               234 non-null object
Title                    3548 non-null object
LastEditorUserId         2815 non-null float64
ParentID                 4672 non-null float64
LastEditDate             2823 non-null object
CommentCount             8438 non-null int64
CommunityOwnedDate       107 non-null object
AnswerCount              3548 non-null float64
AcceptedAnswerId         1223 non-null float64
Score                    8438 non-null int64
OwnerDisplayName         52 non-null object
PostTypeId               8438 non-null int64
OwnerUserId              8415 non-null float64
Tags                     3548 non-null object
CreationDate             8438 non-null object
FavoriteCount            1162 non-null float64
Id                    

In [7]:
post_df.head()

,Body,ViewCount,LastEditorDisplayName,ClosedDate,Title,LastEditorUserId,ParentID,LastEditDate,CommentCount,CommunityOwnedDate,...,AcceptedAnswerId,Score,OwnerDisplayName,PostTypeId,OwnerUserId,Tags,CreationDate,FavoriteCount,Id,LastActivityDate
0,<p>I've always been interested in machine lear...,306,None,2014-05-14T14:40:25.950,How can I do simple machine learning without h...,NaN,NaN,None,1,None,...,NaN,7,None,1,5,<machine-learning>,2014-05-13T23:58:30.457,1,5,2014-05-14T00:36:31.077
1,"<p>As a researcher and instructor, I'm looking...",285,None,2014-05-14T08:40:54.950,What open-source books (or other materials) pr...,97,NaN,2014-05-16T13:45:00.237,4,None,...,10,2,None,1,36,<education><open-source>,2014-05-14T00:11:06.457,1,7,2014-05-16T13:45:00.237
2,"<p>Not sure if this fits the scope of this SE,...",NaN,None,None,None,NaN,5,None,0,None,...,NaN,4,None,2,51,None,2014-05-14T00:36:31.077,NaN,9,2014-05-14T00:36:31.077
3,"<p>One book that's freely available is ""The El...",NaN,None,None,None,NaN,7,None,1,None,...,NaN,10,None,2,22,None,2014-05-14T00:53:43.273,NaN,10,2014-05-14T00:53:43.273
4,<p>I am sure data science as will be discussed...,737,None,None,Is Data Science the Same as Data Mining?,322,NaN,2014-06-17T16:17:20.473,1,None,...,NaN,16,None,1,66,<data-mining><definitions>,2014-05-14T01:25:59.677,3,14,2014-06-20T17:36:05.023


In [8]:
post_df.drop(['LastEditorDisplayName','CommunityOwnedDate','LastEditorUserId','LastEditDate',
             'LastActivityDate'],axis=1,inplace=True)

In [9]:
post_df.columns

Index([u'Body', u'ViewCount', u'ClosedDate', u'Title', u'ParentID',
       u'CommentCount', u'AnswerCount', u'AcceptedAnswerId', u'Score',
       u'OwnerDisplayName', u'PostTypeId', u'OwnerUserId', u'Tags',
       u'CreationDate', u'FavoriteCount', u'Id'],
      dtype='object')

In [10]:
post_df.head()

,Body,ViewCount,ClosedDate,Title,ParentID,CommentCount,AnswerCount,AcceptedAnswerId,Score,OwnerDisplayName,PostTypeId,OwnerUserId,Tags,CreationDate,FavoriteCount,Id
0,<p>I've always been interested in machine lear...,306,2014-05-14T14:40:25.950,How can I do simple machine learning without h...,NaN,1,1,NaN,7,None,1,5,<machine-learning>,2014-05-13T23:58:30.457,1,5
1,"<p>As a researcher and instructor, I'm looking...",285,2014-05-14T08:40:54.950,What open-source books (or other materials) pr...,NaN,4,3,10,2,None,1,36,<education><open-source>,2014-05-14T00:11:06.457,1,7
2,"<p>Not sure if this fits the scope of this SE,...",NaN,None,None,5,0,NaN,NaN,4,None,2,51,None,2014-05-14T00:36:31.077,NaN,9
3,"<p>One book that's freely available is ""The El...",NaN,None,None,7,1,NaN,NaN,10,None,2,22,None,2014-05-14T00:53:43.273,NaN,10
4,<p>I am sure data science as will be discussed...,737,None,Is Data Science the Same as Data Mining?,NaN,1,4,NaN,16,None,1,66,<data-mining><definitions>,2014-05-14T01:25:59.677,3,14


In [11]:
def create_plain_text(html_formatted_text):
    plain_text=BeautifulSoup(html_formatted_text,'html.parser').get_text()
    plain_text = plain_text.replace(u'\u200b','')
    plain_text = plain_text.replace(u'\xa0','')
    return plain_text

In [12]:
post_df['Body_Text']=post_df.apply(lambda k : create_plain_text(k['Body']),axis=1)

In [13]:
post_df.head()

,Body,ViewCount,ClosedDate,Title,ParentID,CommentCount,AnswerCount,AcceptedAnswerId,Score,OwnerDisplayName,PostTypeId,OwnerUserId,Tags,CreationDate,FavoriteCount,Id,Body_Text
0,<p>I've always been interested in machine lear...,306,2014-05-14T14:40:25.950,How can I do simple machine learning without h...,NaN,1,1,NaN,7,None,1,5,<machine-learning>,2014-05-13T23:58:30.457,1,5,I've always been interested in machine learnin...
1,"<p>As a researcher and instructor, I'm looking...",285,2014-05-14T08:40:54.950,What open-source books (or other materials) pr...,NaN,4,3,10,2,None,1,36,<education><open-source>,2014-05-14T00:11:06.457,1,7,"As a researcher and instructor, I'm looking fo..."
2,"<p>Not sure if this fits the scope of this SE,...",NaN,None,None,5,0,NaN,NaN,4,None,2,51,None,2014-05-14T00:36:31.077,NaN,9,"Not sure if this fits the scope of this SE, bu..."
3,"<p>One book that's freely available is ""The El...",NaN,None,None,7,1,NaN,NaN,10,None,2,22,None,2014-05-14T00:53:43.273,NaN,10,"One book that's freely available is ""The Eleme..."
4,<p>I am sure data science as will be discussed...,737,None,Is Data Science the Same as Data Mining?,NaN,1,4,NaN,16,None,1,66,<data-mining><definitions>,2014-05-14T01:25:59.677,3,14,I am sure data science as will be discussed in...


In [14]:
best_ques = post_df[(post_df.PostTypeId==1)].sort_values(['Score','ViewCount','AnswerCount','FavoriteCount','CommentCount'],ascending=[False,False,False,False,False]).head()

In [15]:
best_ques[['Title','Body_Text','Tags','Score']]

,Title,Body_Text,Tags,Score
128,Publicly Available Datasets,One of the common problems in data science is ...,<open-source><dataset>,83
612,Best python library for neural networks,I'm using Neural Networks to solve different M...,<machine-learning><python><neuralnetwork>,75
9,How big is big data?,Lots of people use the term big data in a rath...,<bigdata><scalability><efficiency><performance>,49
12,K-Means clustering for mixed numeric and categ...,My data set contains a number of numeric attri...,<data-mining><clustering><octave><k-means><cat...,45
627,"Are Support Vector Machines still considered ""...",This question is in response to a comment I sa...,<data-mining><svm><state-of-the-art>,37


In [16]:
post_df.Title.fillna('None',inplace=True)
post_df.Title=pd.Series([title for title in post_df.Title if title!=None])

In [17]:
post_df.Tags.fillna('None',inplace=True)

In [18]:
post_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8438 entries, 0 to 8437
Data columns (total 17 columns):
Body                8438 non-null object
ViewCount           3548 non-null float64
ClosedDate          234 non-null object
Title               8438 non-null object
ParentID            4672 non-null float64
CommentCount        8438 non-null int64
AnswerCount         3548 non-null float64
AcceptedAnswerId    1223 non-null float64
Score               8438 non-null int64
OwnerDisplayName    52 non-null object
PostTypeId          8438 non-null int64
OwnerUserId         8415 non-null float64
Tags                8438 non-null object
CreationDate        8438 non-null object
FavoriteCount       1162 non-null float64
Id                  8438 non-null int64
Body_Text           8438 non-null object
dtypes: float64(6), int64(4), object(7)
memory usage: 955.9+ KB


In [19]:
# cleaning tegs
import re
def clean_tag(tags):
    if tags!=None:
        modify_tags = re.sub('<|>',' ',str(tags)).split()
        return modify_tags
    else:
        return 

In [20]:
post_df.Tags=post_df.Tags.map(lambda r : clean_tag(r))

In [21]:
post_df['Tags'].head()

0            [machine-learning]
1      [education, open-source]
2                        [None]
3                        [None]
4    [data-mining, definitions]
Name: Tags, dtype: object

In [22]:
post_df[['Title','Body_Text','Tags','Score']].head(10)

,Title,Body_Text,Tags,Score
0,How can I do simple machine learning without h...,I've always been interested in machine learnin...,[machine-learning],7
1,What open-source books (or other materials) pr...,"As a researcher and instructor, I'm looking fo...","[education, open-source]",2
2,None,"Not sure if this fits the scope of this SE, bu...",[None],4
3,None,"One book that's freely available is ""The Eleme...",[None],10
4,Is Data Science the Same as Data Mining?,I am sure data science as will be discussed in...,"[data-mining, definitions]",16
5,What are the advantages and disadvantages of S...,In which situations would one system be prefer...,[databases],2
6,Use liblinear on big data for semantic analysis,I use Libsvm to train data and predict classif...,"[machine-learning, bigdata, libsvm]",15
7,None,LIBSVM is a library for support vector classif...,[None],0
8,None,,[None],0
9,How big is big data?,Lots of people use the term big data in a rath...,"[bigdata, scalability, efficiency, performance]",49


In [23]:
train_data=post_df['Title'].values.tolist()

In [24]:
train_data=[data for data in train_data if data!=None]

In [25]:
# Answer
best_answer=post_df[(post_df.PostTypeId==2)].sort(['Score'],ascending=[False]).head(100)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [26]:
best_answer[['Score','Body_Text']].head()

,Score,Body_Text
613,69,UPDATE: the landscape has changed quite a bit ...
24,61,To me (coming from a relational database backg...
131,46,"There is, in fact, a very reasonable list of p..."
8126,44,So you can integrate with the rest of the code...
298,40,Some real important differences to consider wh...


### import and instatiate count Vectorizer
#### Count vectorizer convert text in to matrix of token count

In [27]:
from sklearn.feature_extraction.text import CountVectorizer

In [28]:
# now Learning vocabulury of training data
# vectorizer.fit(train_data)

In [29]:
# examinine fitted vocabolury
# vectorizer.get_feature_names()[:10]

In [30]:
# transforming training data into document matrix
# X=vectorizer.transform(train_data)


In [31]:
# converting sparse matrix to dense matrix
# X.toarray()

In [32]:
# examine the vocabulary and document-term matrix together
# pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names()).head()

In [33]:
post_df[['Tags']].head()

,Tags
0,[machine-learning]
1,"[education, open-source]"
2,[None]
3,[None]
4,"[data-mining, definitions]"


In [34]:
def process_tag(tags):
    tag_str_format=""
    if tags!=None:
        for i in range(len(tags)-1):
            tag_str_format += tags[i] + ","
        tag_str_format += tags[len(tags)-1]
        return tag_str_format
    else:
        return 
            

In [35]:
post_df['Tags_Target']=post_df.Tags.map(lambda tag : process_tag(tag))

In [36]:
from sklearn import preprocessing
le=preprocessing.LabelEncoder()

In [37]:
tag_list= []          # contains values of d_set['tag']
tag_list=[tag for tag in post_df.Tags if tag!=None]
tag_list = [sub_tag for tag in tag_list for sub_tag in tag]
tag_list=list(set(tag_list))

le.fit(tag_list)

post_df['Tags_label_num']=pd.Series([le.transform(tag) for tag in post_df.Tags if tag!=None])

post_df[['Title','Tags','Tags_label_num','Tags_Target']].head(10)

,Title,Tags,Tags_label_num,Tags_Target
0,How can I do simple machine learning without h...,[machine-learning],[151],machine-learning
1,What open-source books (or other materials) pr...,"[education, open-source]","[77, 184]","education,open-source"
2,None,[None],[1],None
3,None,[None],[1],None
4,Is Data Science the Same as Data Mining?,"[data-mining, definitions]","[60, 69]","data-mining,definitions"
5,What are the advantages and disadvantages of S...,[databases],[64],databases
6,Use liblinear on big data for semantic analysis,"[machine-learning, bigdata, libsvm]","[151, 28, 146]","machine-learning,bigdata,libsvm"
7,None,[None],[1],None
8,None,[None],[1],None
9,How big is big data?,"[bigdata, scalability, efficiency, performance]","[28, 227, 78, 195]","bigdata,scalability,efficiency,performance"


In [38]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb=MultiLabelBinarizer()

In [39]:
trained_tags_label=[tag_num for tag_num in post_df.Tags_label_num ]
len(trained_tags_label)

8438

In [40]:
mlb.fit(trained_tags_label)

MultiLabelBinarizer(classes=None, sparse_output=False)

In [41]:
trained_tags_dtm=mlb.transform(trained_tags_label)
trained_tags_dtm

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

### Vectorize dataset

In [44]:
# instatiate the vectorizer
vect_for_title=CountVectorizer(max_df=0.5, min_df=1, stop_words='english')

In [45]:
# Learn train_data from vocabulury
vect_for_title.fit(train_data)

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=0.5, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [46]:
X_train_dtm=vect_for_title.transform(train_data)

In [47]:
X_train_array=X_train_dtm.toarray()

In [51]:
X_train_array[:2]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [56]:
df=pd.DataFrame(X_train_array,columns=vect_for_title.get_feature_names())
df.loc[0:2,:]

,04,10,100,1000,100000,1070,11,15,16,16s,...,yearprediction,years,yes,yielding,yields,zeppelin,zero,zeror,zip,zookeeper
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
X=X_train_array
y=trained_tags_dtm

In [84]:
from sklearn.cross_validation import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [85]:
X_train.shape, X_test.shape,y_train.shape, y_test.shape

((6750, 4027), (1688, 4027), (6750, 283), (1688, 283))

In [83]:
from sklearn.naive_bayes import MultinomialNB
nb=MultinomialNB()
nb.fit(X_train,y_train)

ValueError: bad input shape (6328, 283)